In [3]:
import io
import json
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [4]:
def write_to_table(df,schema,table_name,password='1q2w3e4r',host='127.0.0.1',DB='RENT',if_exists='append',port='5432'):
    db_engine = create_engine('postgresql://postgres:{}@{}:{}/{}'.format(password,host,port,DB))
    string_data_io = io.StringIO()
    df.to_csv(string_data_io,sep=',',index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name,pd_sql_engine,frame=df,index=False,if_exists=if_exists,schema=schema)
    table.create()
    
    string_data_io.seek(0)
    string_data_io.readline()
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = '''COPY "%s"."%s" FROM STDIN HEADER DELIMITER ',' CSV''' % (schema, table_name)
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [5]:
conn = psycopg2.connect(database='ESF',user='postgres',password='1q2w3e4r',host='127.0.0.1',port=5432)

In [6]:
with open("bk_city_map.json", 'r', encoding='utf-8') as fp:
    city_dist = json.loads(fp.read())
city_list = []
for city, districts in city_dist.items():
    city_list.append(city)

In [5]:
city_list

['合肥',
 '芜湖',
 '马鞍山',
 '安庆',
 '北京',
 '重庆',
 '福州',
 '厦门',
 '泉州',
 '漳州',
 '广州',
 '深圳',
 '珠海',
 '佛山',
 '江门',
 '湛江',
 '惠州',
 '东莞',
 '中山',
 '清远',
 '南宁',
 '柳州',
 '桂林',
 '北海',
 '防城港',
 '贵阳',
 '遵义',
 '兰州',
 '石家庄',
 '唐山',
 '邯郸',
 '保定',
 '廊坊',
 '张家口',
 '哈尔滨',
 '郑州',
 '开封',
 '洛阳',
 '新乡',
 '许昌',
 '信阳',
 '周口',
 '驻马店',
 '武汉',
 '黄石',
 '宜昌',
 '襄阳',
 '鄂州',
 '孝感',
 '黄冈',
 '长沙',
 '株洲',
 '衡阳',
 '岳阳',
 '常德',
 '永州',
 '海口',
 '三亚',
 '澄迈',
 '长春',
 '吉林',
 '南京',
 '无锡',
 '徐州',
 '常州',
 '苏州',
 '昆山',
 '南通',
 '连云港',
 '淮安',
 '盐城',
 '镇江',
 '海门',
 '江阴',
 '南昌',
 '景德镇',
 '萍乡',
 '九江',
 '新余',
 '赣州',
 '吉安',
 '宜春',
 '抚州',
 '上饶',
 '沈阳',
 '大连',
 '丹东',
 '呼和浩特',
 '包头',
 '赤峰',
 '银川',
 '太原',
 '大同',
 '运城',
 '晋中',
 '济南',
 '青岛',
 '淄博',
 '烟台',
 '潍坊',
 '济宁',
 '泰安',
 '威海',
 '临沂',
 '菏泽',
 '上海',
 '成都',
 '泸州',
 '绵阳',
 '遂宁',
 '内江',
 '南充',
 '眉山',
 '宜宾',
 '达州',
 '巴中',
 '凉山',
 '乐山',
 '广元',
 '西安',
 '宝鸡',
 '咸阳',
 '汉中',
 '天津',
 '乌鲁木齐',
 '昆明',
 '大理',
 '杭州',
 '宁波',
 '温州',
 '嘉兴',
 '湖州',
 '绍兴',
 '金华',
 '义乌',
 '台州']

In [7]:
for city in city_list:
    sql = f"""SELECT id, "城市", "区县", "标题url", "小区", "地址", "数据来源", "单价", "总价", "户型", "面积", "楼层", "建筑年份", "朝向", "抓取时间", "抓取年份", "抓取月份","关注人数","标签"
        FROM public."Esf_2020" where 城市='{city}';"""
    df_id = pd.read_sql_query(sql, con=conn)
    # print(df_id.shape)
    # df_id.drop_duplicates(['标题', '标题url'], inplace=True)
    print(df_id.shape)
    write_to_table(df_id, schema='public', table_name='Esf_2020',password='1q2w3e4r', host='192.168.88.254',DB='ESF' ,port='15432')
    

(66253, 19)
(27793, 19)
(12013, 19)
(20674, 19)
(70096, 19)
(75676, 19)
(29962, 19)
(33180, 19)
(21808, 19)
(13533, 19)
(65823, 19)
(42104, 19)
(15677, 19)
(29726, 19)
(15373, 19)
(12319, 19)
(33075, 19)
(54904, 19)
(67384, 19)
(8441, 19)
(32935, 19)
(18863, 19)
(23383, 19)
(12168, 19)
(6335, 19)
(34168, 19)
(10549, 19)
(19081, 19)
(43901, 19)
(32728, 19)
(12248, 19)
(15126, 19)
(34117, 19)
(27107, 19)
(36773, 19)
(71400, 19)
(11635, 19)
(23151, 19)
(14999, 19)
(11852, 19)
(12935, 19)
(20386, 19)
(6009, 19)
(80537, 19)
(14581, 19)
(22238, 19)
(17366, 19)
(12368, 19)
(7226, 19)
(5851, 19)
(38280, 19)
(16288, 19)
(7640, 19)
(9560, 19)
(7856, 19)
(5645, 19)
(11066, 19)
(2392, 19)
(657, 19)
(59414, 19)
(18543, 19)
(60072, 19)
(36171, 19)
(31528, 19)
(25810, 19)
(41999, 19)
(46706, 19)
(43941, 19)
(8667, 19)
(19047, 19)
(16661, 19)
(21401, 19)
(6299, 19)
(8529, 19)
(57889, 19)
(3577, 19)
(4054, 19)
(17637, 19)
(3539, 19)
(11124, 19)
(17629, 19)
(5999, 19)
(5086, 19)
(12042, 19)
(33405, 19)


In [7]:
sql = f"""SELECT id, "城市", "区县", "标题url", "小区", "地址", "数据来源", "单价", "总价", "户型", "面积", "楼层", "建筑年份", "朝向", "抓取时间", "抓取年份", "抓取月份","关注人数","标签"
        FROM public."Esf_2020";"""
df_id = pd.read_sql_query(sql, con=conn)
df_id.shape()

TypeError: 'tuple' object is not callable